In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_DESA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon = individual
    if C <= 0 or epsilon <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
parameter_ranges = [(0.0001, 0.1), (0.001, 1)]

# Register genetic operators
toolbox.register("attr_float", random.uniform)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (lambda: random.uniform(*parameter_ranges[0]),
                  lambda: random.uniform(*parameter_ranges[1])), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 80
population = toolbox.population(n=population_size)

# Genetic algorithm parameters
crossover_rate = 0.3
mutation_rate = 0.7
num_generations = 100

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon = best_individual

# Test the best SVR model
best_svr = SVR(kernel='linear', C=C, epsilon=epsilon)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu: [0.07421435014795592, 0.5492014819562437], MAPE: 57.69795835972307
Individu: [0.08646616374128294, 0.3472913022228308], MAPE: 58.999581626221286
Individu: [0.02083509625027538, 0.9496905823430692], MAPE: 57.69795835972307
Individu: [0.005424367383967985, 0.8381237697994438], MAPE: 57.69795835972307
Individu: [0.05289027238233302, 0.7430852382495249], MAPE: 57.69795835972307
Individu: [0.033401048129759386, 0.7634197875639096], MAPE: 57.69795835972307
Individu: [0.07391609635055342, 0.09411286155518578], MAPE: 57.56233191136716
Individu: [0.03913480069431796, 0.0], MAPE: inf
Individu: [0.009143991075370481, 0.5507355671918188], MAPE: 57.69795835972307
Individu: [0.05319489458228121, 0.7352084901029807], MAPE: 57.69795835972307
Individu: [0.015603539164537402, 0.13248818002258794], MAPE: 59.02459572375489
Individu: [0.08933343789689943, 0.26537019680304136], MAPE: 61.28543890023464
Individu: [0.0, 0.2432910590022459], MAPE: inf
Individu: [0.09212070758149939, 0.8808

In [6]:
# Nilai minimum dan maksimum asli dari data yang digunakan untuk normalisasi
data_min = 0  # Sesuaikan dengan nilai minimum data asli Anda
data_max = 20  # Sesuaikan dengan nilai maksimum data asli Anda

# Denormalisasi secara manual
y_pred_denormalized = y_pred * (data_max - data_min) + data_min

#pred_denormalized =  MinMaxScaler.inverse_transform(y_pred.reshape(1, -1))
print(y_pred_denormalized)

[ 9.99848668 10.09083231  9.2597216   8.7056478   8.79799343  9.99848668
  9.07503034  9.81379541  9.44441287  8.52095653  9.99848668  9.53675851]
